In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pprint
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support import expected_conditions as EC


import demjson3
import os
import json

import sys
sys.path.append('/full/path/to/web-scraping')



from salaries_functions import create_country_file, write_jobs_to_txt, scrape_pages, parse_response_to_csv, get_netherlands_url, get_dataframe, convert_cols_to_float

os.chdir('../database')
from crud_functions import write_dataframes_to_db
from models import Software_Engineer

os.chdir('../salaries')

In [ ]:
netherlands_url = get_netherlands_url()
response = requests.get(netherlands_url)

In [ ]:
germany_url = 'https://techpays.com/europe/germany'
response = requests.get(germany_url)

In [ ]:
netherlands_url = 'https://techpays.com/europe/netherlands'
response = requests.get(netherlands_url)
create_country_file('netherlands')
create_country_file('germany')

In [ ]:
scrape_pages(netherlands_url)
scrape_pages(germany_url)

earlier netherlands iteration crashed at 129th iteration

berlin search has to stop at the 107th iteration! (added break statement)

In [ ]:
parse_response_to_csv('berlin')
parse_response_to_csv('netherlands')

In [ ]:
for title in dutch_salaries['title'].dropna().unique():
    if 'senior' in title:
        print(title)
    if 'medior' in title:
        print(title)
    if 'junior' in title:
        print(title)

In [ ]:

def get_page_filter_button_types_dict():
    arguments = dict.fromkeys(('locationFilterButton','jobFamilyFilterButton', 'seniorityFilterButton','roleTypeFilterButton','companyFilterButton'))
    return arguments


def get_filter_button_values(driver, filter_button):
    '''
    This functions receives a WebDriver object and a button id Object
    It returns a dictionary of all possible values for that button.
    '''
    options = {}
    button_class = re.sub(r'Button', '', filter_button) + 'Options'
    print(f'Filter button is {filter_button}, and button class is {button_class}')

    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, filter_button))
    )
    
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
    
    
    
    button.click()


    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, button_class))
    )

    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    dropdown = soup.find("div", id= button_class)
    locations = dropdown.find_all("a")

    for loc in locations:
        name = loc.text.strip()
        link = "https://techpays.com" + loc["href"]
        #print(f"{name}: {link}")
        options[name] = link
    button.send_keys(Keys.SPACE)
    return options





#########   MAIN    ####
driver = webdriver.Chrome()
driver.get(netherlands_url)
button_ids = get_page_filter_button_types_dict()
button_id_values = {button_id: {} for button_id in button_ids}

for button_id in button_ids:
    button_id_values = get_filter_button_values(driver, button_id)
    button_ids[button_id] = button_id_values
    print(button_ids[button_id])

    
    
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
    driver.execute_script("document.querySelector('body').click();")
    time.sleep(1) 




In [ ]:
print(json.dumps({button_id: button_ids[button_id]}, indent=2, ensure_ascii=False))


In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get('/')
async def read_root():
    return 'Hello World'

In [ ]:

os.chdir('../database')
from models import software_engineer

os.chdir('../salaries')

-
from models import software_engineer


In [ ]:
os.chdir('../salaries')

In [ ]:

os.chdir('../database')
from crud_functions import write_dataframes_to_db
from models import software_engineer

In [2]:
df = get_dataframe('berlin')
df = convert_cols_to_float(df)
df['country'] = 'Germany'

write_dataframes_to_db(df, Software_Engineer)

newa
title                        object
guid                         object
specialization               object
city                         object
companyName                  object
totalCompensation           float64
totalCompensationNumber       int64
totalCompensationDetails     object
baseSalary                  float64
baseSalaryNumber              int64
oldYearForData              float64
otherContext                 object
dtype: object
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000177B5D509B0>, 'title': 'Senior Software Engineer ', 'guid': '7wlsz77oodzc75obmli2a', 'specialization': 'Backend', 'city': 'nan', 'companyName': 'Delivery Hero', 'totalCompensation': nan, 'totalCompensationNumber': 120000, 'totalCompensationDetails': '€100K salary, €20K equity/year', 'baseSalary': nan, 'baseSalaryNumber': 100000, 'oldYearForData': nan, 'otherContext': 'nan', 'country': 'Germany', 'id': None}


In [3]:
x = get_dataframe('Netherlands')
x['country'] = 'The Netherlands'

write_dataframes_to_db(x, Software_Engineer)

newa
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000177B67ABD70>, 'title': 'Fullstack Software Engineer', 'guid': 'd5m23tirj1tmu1d4lalf8', 'specialization': 'nan', 'city': 'Arnhem (flexible days)', 'companyName': 'Avisi', 'totalCompensation': nan, 'totalCompensationNumber': 56000, 'totalCompensationDetails': '€56K salary, €1K benefits', 'baseSalary': nan, 'baseSalaryNumber': 56000, 'oldYearForData': nan, 'otherContext': 'nan', 'country': 'The Netherlands', 'id': None}


In [ ]:
germany_from_fyi = scrape_from_fyi('Germany')